#### 주가 예측 실습

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

saveDir = './Saver/predictStock(reg).save'   # 학습 결과를 저정할 폴더
saveFile = saveDir + '/save'   # 학습 결과를 저장할 파일

In [2]:
sm = pd.read_csv('./stockData/005930.csv', index_col=0, parse_dates=True)
sm = sm.sort_index() # 날짜 순으로 정렬한다
sm = sm.dropna()
sm = sm / 10000

In [3]:
sm.head()

,open,high,low,close,volume
date,,,,,
2011-05-12,1.772,1.802,1.766,1.770,55.2942
2011-05-13,1.792,1.848,1.780,1.832,66.9154
2011-05-16,1.834,1.838,1.796,1.802,32.2905
2011-05-17,1.794,1.796,1.778,1.780,23.1293
2011-05-18,1.786,1.798,1.772,1.786,33.1824


In [4]:
sm.tail()

,open,high,low,close,volume
date,,,,,
2019-06-19,4.545,4.545,4.500,4.535,1041.7690
2019-06-20,4.485,4.550,4.485,4.550,714.7785
2019-06-21,4.575,4.580,4.520,4.570,945.4913
2019-06-24,4.520,4.580,4.520,4.550,608.5066
2019-06-25,4.520,4.580,4.520,4.545,412.0726


In [5]:
sm['next close'] = sm['close'].shift(-1)
sm = sm.dropna()

In [6]:
sm['class'] = 0
sm.loc[sm['next close'] > sm['close'], 'class'] = 1

In [7]:
sm = sm.sample(frac=1)
sm.head()

,open,high,low,close,volume,next close,class
date,,,,,,,
2019-05-15,4.270,4.305,4.255,4.255,767.0185,4.155,0
2017-04-18,4.168,4.182,4.128,4.150,13.7213,4.090,0
2018-02-12,4.510,4.632,4.504,4.572,31.5099,4.754,1
2017-05-08,4.552,4.702,4.534,4.702,39.1651,4.560,0
2018-10-08,4.420,4.520,4.420,4.495,675.9464,4.530,1


In [8]:
len(sm.loc[sm['class'] == 0])

993

In [9]:
sm.head()

,open,high,low,close,volume,next close,class
date,,,,,,,
2019-05-15,4.270,4.305,4.255,4.255,767.0185,4.155,0
2017-04-18,4.168,4.182,4.128,4.150,13.7213,4.090,0
2018-02-12,4.510,4.632,4.504,4.572,31.5099,4.754,1
2017-05-08,4.552,4.702,4.534,4.702,39.1651,4.560,0
2018-10-08,4.420,4.520,4.420,4.495,675.9464,4.530,1


In [10]:
inputX = np.array(sm.iloc[:1500, 0:4])
outputY = np.array(sm.iloc[:1500, 6]).reshape(-1,1)

testX = np.array(sm.iloc[1500:, 0:4])
testY = np.array(sm.iloc[1500:, 6]).reshape(-1,1)

In [11]:
nInput = 4      # input layer의 neuron 개수
nHidden1 = 60   # hidden layer-1의 neuron 개수
nHidden2 = 60   # hidden layer-2의 neuron 개수 
nOutput = 1     # output layer의 neuron 개수

In [12]:
# 그래프를 생성한다
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape=[None, nInput], name='x')
y = tf.placeholder(tf.float32, shape=[None, nOutput], name='y')

# hidden layer의 Weight (Wh)와 Bias (Bh)
Wh1 = tf.Variable(tf.truncated_normal([nInput, nHidden1]), dtype=tf.float32, name='Wh1')
Bh1 = tf.Variable(tf.zeros(nHidden1), dtype=tf.float32, name='Bh1')

# hidden layer-2의 Weight (Wh)와 Bias (Bh)
Wh2 = tf.Variable(tf.truncated_normal([nHidden1, nHidden2]), dtype=tf.float32, name='Wh2')
Bh2 = tf.Variable(tf.zeros(nHidden2), dtype=tf.float32, name='Bh2')

# output layer의 Weight (Wo)와 Bias (Bo)
Wo = tf.Variable(tf.truncated_normal([nHidden2, nOutput]), dtype=tf.float32, name='Wo')
Bo = tf.Variable(tf.zeros(nOutput), dtype=tf.float32, name='Bo')

# Hidden layer의 출력값. activation function은 sigmoid
H1 = tf.nn.relu(tf.matmul(x, Wh1) + Bh1, name='H1')
H2 = tf.nn.relu(tf.matmul(H1, Wh2) + Bh2, name='H2')

# Output layer의 출력값. activation function은 sigmoid
predY = tf.nn.sigmoid(tf.add(tf.matmul(H2, Wo), Bo, name='predY'))

# Cost function 정의. MSE
cost = tf.reduce_mean(tf.square(tf.subtract(predY, y)))

# 학습
alpha = tf.placeholder(tf.float32, shape=(), name='alpha')
optimizer = tf.train.AdamOptimizer(alpha)
train = optimizer.minimize(cost)

saver = tf.train.Saver()

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
# 그래프를 실행한다
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 기존 학습 결과를 적용한 후 추가로 학습한다
if tf.train.checkpoint_exists(saveDir):
    saver.restore(sess, saveFile)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./Saver/predictStock(reg).save/save


In [14]:
# Mini-batch 적용
nBatchCnt = 5
nBatchSize = int(inputX.shape[0] / nBatchCnt)
for i in range(10000):
    # Mini-batch 방식으로 학습한다
    for n in range(nBatchCnt):
        # input 데이터를 Mini-batch 크기에 맞게 자른다
        nFrom = n * nBatchSize
        nTo = n * nBatchSize + nBatchSize

        # 마지막 루프이면 nTo는 input 데이터의 끝까지.
        if n == nBatchCnt - 1:
            nTo = inputX.shape[0]
            
        bx = inputX[nFrom : nTo]
        by = outputY[nFrom : nTo]
        
        _, xcost = sess.run([train, cost], feed_dict={x: bx, y: by, alpha:0.001})
    
    if i % 1000 == 0:
        print('cost = %.6f' % xcost)
        
# 학습 결과를 저장해 둔다
saver.save(sess, saveFile)

cost = 0.250897
cost = 0.250049
cost = 0.250498
cost = 0.250226
cost = 0.250262
cost = 0.250233
cost = 0.249852
cost = 0.249671
cost = 0.249399
cost = 0.249063


'./Saver/predictStock(reg).save/save'

In [15]:
testX

array([[2.6  , 2.622, 2.582, 2.604],
       [3.028, 3.052, 2.966, 2.984],
       [2.564, 2.6  , 2.564, 2.598],
       ...,
       [2.658, 2.698, 2.64 , 2.686],
       [2.662, 2.662, 2.61 , 2.614],
       [1.452, 1.472, 1.42 , 1.454]])

In [16]:
# 학습이 완료되면, Wh, Bh, Wo, Bo 이 모두 업데이트 되었으면, inputX를 넣어서 출력값을 확인한다.
# 출력값은 inputY의 추정값이다.
yHat = sess.run(predY, feed_dict={x: testX})

In [17]:
# yHat[np.where(yHat > 0.5)] = 1
# yHat[np.where(yHat < 0.5)] = 0

yHat = np.array([1 if  x > 0.5 else 0 for x in yHat]).reshape(-1,1)
                                        #하나씩! 돌려

In [18]:
accuracy = (testY == yHat).sum() / len(testX)
# 다 더해버리면 true가 몇개인지, 
 
print("Accuracy = %.4f" % accuracy)

Accuracy = 0.5271


# 피쳐 늘려야 할 필요성